# Data Processing For Fulton County Sales and Parcel Data

### Helper Functions
---

In [1]:
import os
import polars as pl
import re

pl.enable_string_cache(True)
pl.Config.set_tbl_cols = 200

In [2]:
def read_file(path: str, dtypes: dict, new_names: list = []) -> pl.DataFrame:
    print("Reading file: ", path)
    df = pl.DataFrame()
    columns = [x for x in dtypes.keys()]
    name_mapping = dict(zip(columns, new_names))

    if path[-4:len(path)] == "xlsx":
        df = pl.read_excel(path,
                           xlsx2csv_options= {"ignore_formats": ["date", "float"], "infer_schema_length": 0},
                           read_csv_options= {"infer_schema_length": 0, "columns": columns, "dtypes": dtypes}
        )
    else:
        df = pl.read_csv(path,
                         separator= '\t',
                         columns= columns,
                         dtypes= dtypes)
    
    if new_names:
        df = df.rename(name_mapping)

    return df

def strip_parid(df: pl.DataFrame) -> pl.DataFrame:
    df = df.with_columns(
        pl.col('parid').str.replace_all(' ', '').alias('parid_strip')
    )
    return df


In [3]:
##############################
# SET PRE-PROCESSING VARIABLES
##############################
parcels_path = "C:/Users/nicho/Documents/research/FCS/data/digest/"
sales_path = "C:/Users/nicho/Documents/research/FCS/data/sales/"
parcel_dtypes = {
    'Taxyr': pl.Categorical, 'Parid': pl.Utf8, 'Nbhd': pl.Categorical,  
    'Situs Adrno': pl.Int64, 'Situs Adrdir': pl.Utf8, 
    'Situs Adrstr': pl.Utf8, 'Situs Adrsuf': pl.Utf8, 
    'Situs Adrsuf2': pl.Utf8, 'Cityname': pl.Categorical, 
    'Zoning': pl.Categorical, 'Muni': pl.Categorical, 'Class': pl.Categorical, 
    'Luc': pl.Categorical, 'Livunit': pl.Utf8, 'Calcacres': pl.Utf8, 
    'Note1': pl.Utf8, 'Taxdist': pl.Categorical, 'Own1': pl.Utf8, 
    'Own2': pl.Utf8, 'Owner Adrno': pl.Utf8, 
    'Owner Adradd': pl.Utf8, 'Owner Adrdir': pl.Categorical, 
    'Owner Adrstr': pl.Utf8, 'Owner Adrsuf': pl.Categorical, 
    'Owner Adrsuf2': pl.Utf8, 'Statecode': pl.Categorical, 
    'Country': pl.Categorical, 'Unitno': pl.Utf8, 'Zip1': pl.Utf8, 
    'Aprland': pl.Int32, 'Aprbldg': pl.Int32, 'Style': pl.Categorical, 
    'D Yrblt': pl.Categorical, 'Rmtot': pl.Int16, 'Rmbed': pl.Int16, 
    'Fixbath': pl.Int16, 'Fixhalf': pl.Int16
}
parcel_new_names = [
    'taxyr', 'parid', 'nbhd', 'site_adrno', 'site_adrdir',
    'site_adrstr', 'site_adrsuf', 'site_adrsuf2', 'site_cityname',
    'zoning', 'site_muni', 'class', 'luc', 'livunit', 'calcacres',
    'note1', 'taxdist', 'own1', 'own2', 'own_adrno', 'own_adradd',
    'own_adrdir', 'own_adrstr', 'own_adrsuf',
    'own_adrsuf2', 'own_statecode', 'own_country', 'own_unitno',
    'own_zip', 'aprland', 'aprbldg', 'style', 'yrblt',
    'rmtot', 'rmbed', 'fixbath', 'fixhalf'
]
sales_dtypes = {
    'Taxyr': pl.Categorical, 'Saledt: Year (YYYY)': pl.Categorical,
    'Saledt: Month (Mon)': pl.Categorical, 'Parid': pl.Utf8,
    'Saledt': pl.Utf8, 'SALES PRICE': pl.Utf8, 
    'FAIR MARKET VALUE': pl.Utf8, 'DEED TYPE': pl.Categorical,
    'Costval': pl.Utf8, 'Saleval': pl.Categorical, 'Who': pl.Utf8,
    'Wen': pl.Utf8, 'GRANTOR': pl.Utf8, 'GRANTEE': pl.Utf8,
    'Adrpre':pl.Categorical, 'Adrno': pl.Utf8,
    'Adrdir': pl.Categorical, 'Adrstr': pl.Utf8,
    'Adrsuf': pl.Categorical, 'Adrsuf2': pl.Categorical,
    'Cityname': pl.Categorical, 'Unitno': pl.Utf8,
    'Livunit': pl.Int16
}
sales_new_names = [
    'taxyr', 'sale_year', 'sale_month', 'parid',
    'sale_date', 'salesprice', 'fmv', 'deed',
    'costval', 'saleval', 'appraiser', 'when', 'grantor',
    'grantee', 'sale_adrpre', 'sale_adrno', 'sale_adrdir',
    'sale_adrstr', 'sale_adrsuf', 'sale_adrsuf2',
    'sale_cityname', 'sale_unitno', 'livunit'
]

### Processing Procedure
---

Read all files to join; verify file groupings by year are as expected.

In [4]:
parcel_files = os.listdir(parcels_path)
sales_files = os.listdir(sales_path)
files_by_year = {}

for yr in range(2011, 2023):
    files_by_year[yr] = [sales_path + file for file in sales_files if str(yr) in file] \
                    + [parcels_path + file for file in parcel_files if str(yr) in file]
    
[files_by_year[x] for x in files_by_year][len(files_by_year)-2:]

[['C:/Users/nicho/Documents/research/FCS/data/sales/Sales2021.txt',
  'C:/Users/nicho/Documents/research/FCS/data/digest/parcel_14_2021.xlsx',
  'C:/Users/nicho/Documents/research/FCS/data/digest/parcel_17_2021.xlsx',
  'C:/Users/nicho/Documents/research/FCS/data/digest/parcel_atl_2021.xlsx',
  'C:/Users/nicho/Documents/research/FCS/data/digest/parcel_nf_2021.xlsx',
  'C:/Users/nicho/Documents/research/FCS/data/digest/parcel_sf_2021.xlsx'],
 ['C:/Users/nicho/Documents/research/FCS/data/sales/Sales2022.txt',
  'C:/Users/nicho/Documents/research/FCS/data/digest/parcel_14_2022.xlsx',
  'C:/Users/nicho/Documents/research/FCS/data/digest/parcel_17_2022.xlsx',
  'C:/Users/nicho/Documents/research/FCS/data/digest/parcel_atl_2022.xlsx',
  'C:/Users/nicho/Documents/research/FCS/data/digest/parcel_nf_2022.xlsx',
  'C:/Users/nicho/Documents/research/FCS/data/digest/parcel_sf_2022.xlsx']]

Procedure

- Merge Sales with Their Associated Parcel Info
- Look if there are duplicates within appended parcel data for each year
- Make an Appended Sales
- Make an Appended Parcel?
- All Parcel?

In [5]:
def merge_year(
    sales: str,
    parcel: list,
    merge_type: bool,
    save_steps: bool = True
) -> pl.DataFrame:
    '''Merges sales to parcel files for a given year
    
    Keyword arguments:
    sales: path to sales file
    parcel: path to parcel file
    merge_type: True if left join on sales, False if left join on parcel
    save_steps: True if steps should be saved to a CSV
    '''
    sale_yr = re.findall(r'\d+', sales)[0]
    sales_df = read_file(sales, sales_dtypes, sales_new_names)
    parcel_dfs = [read_file(file, parcel_dtypes, parcel_new_names) for file in parcel]
    parcel_append = pl.concat(parcel_dfs)

    parcel_init_len = len(parcel_append)

    parcel_append = parcel_append.unique()

    sales_df = strip_parid(sales_df)
    parcel_append = strip_parid(parcel_append)

    if merge_type:
        merged = sales_df.join(parcel_append, how='left', on=['parid_strip', 'taxyr'])
    else:
        merged = parcel_append.join(sales_df, how='left', on=['parid_strip', 'taxyr'])

    if save_steps:
        parcel_append.write_csv('../output/parcel/parcels_appended_' + sale_yr + '.csv')
        merged.write_csv('../output/merged/parcel_sales' + sale_yr + '.csv')

    parcel_final_len = len(parcel_append)

    print("Original size of sales: ", len(sales_df))
    print("Original size of parcels: ", parcel_init_len)
    print('Additional rows generated (merged - sales): ', len(merged) - len(sales_df))
    print('Duplicate rows in parcel data: ', parcel_init_len - parcel_final_len)

    unmatched = merged.filter(pl.col('parid_right').is_null())['parid'].to_list()
    error_path = r'../output/errors/merged_unmatched_' + sale_yr + '.csv'
    with open(error_path, 'w') as writer:
        for parid in unmatched:
            writer.write("%s\n" % parid)

    print("Wrote unmatched Parcel IDs to: ", error_path)
    print('Done with year ', sale_yr)

    return merged

In [6]:
sales_left_parcel = [merge_year(files[0], files[1:], True) for files in files_by_year.values()]

Reading file:  C:/Users/nicho/Documents/research/FCS/data/sales/Sales2011.txt
Reading file:  C:/Users/nicho/Documents/research/FCS/data/digest/parcel_14_2010-2011.xlsx
Reading file:  C:/Users/nicho/Documents/research/FCS/data/digest/parcel_17_2010-2011.xlsx
Reading file:  C:/Users/nicho/Documents/research/FCS/data/digest/parcel_atl_2011-2012.xlsx
Reading file:  C:/Users/nicho/Documents/research/FCS/data/digest/parcel_nf_2010-2011.xlsx
Reading file:  C:/Users/nicho/Documents/research/FCS/data/digest/parcel_sf_2010-2011.xlsx
Original size of sales:  47806
Original size of parcels:  941997
Additional rows generated (merged - sales):  281
Duplicate rows in parcel data:  136384
Wrote unmatched Parcel IDs to:  ../output/errors/merged_unmatched_2011.csv
Done with year  2011
Reading file:  C:/Users/nicho/Documents/research/FCS/data/sales/Sales2012.txt
Reading file:  C:/Users/nicho/Documents/research/FCS/data/digest/parcel_14_2012-2013.xlsx
Reading file:  C:/Users/nicho/Documents/research/FCS/d

In [7]:
full_sales_parcel = pl.concat(sales_left_parcel, how='diagonal')

Verify data looks as expected

In [8]:
full_sales_parcel.sample(4)

taxyr,sale_year,sale_month,parid,sale_date,salesprice,fmv,deed,costval,saleval,appraiser,when,grantor,grantee,sale_adrpre,sale_adrno,sale_adrdir,sale_adrstr,sale_adrsuf,sale_adrsuf2,sale_cityname,sale_unitno,livunit,parid_strip,parid_right,nbhd,site_adrno,site_adrdir,site_adrstr,site_adrsuf,site_adrsuf2,site_cityname,zoning,site_muni,class,luc,livunit_right,calcacres,note1,taxdist,own1,own2,own_adrno,own_adradd,own_adrdir,own_adrstr,own_adrsuf,own_adrsuf2,own_statecode,own_country,own_unitno,own_zip,aprland,aprbldg,style,yrblt,rmtot,rmbed,fixbath,fixhalf
cat,cat,cat,str,str,str,str,cat,str,cat,str,str,str,str,cat,str,cat,str,cat,cat,cat,str,i16,str,str,cat,i64,str,str,str,str,cat,cat,cat,cat,cat,str,str,str,cat,str,str,str,str,cat,str,cat,str,cat,cat,str,str,i32,i32,cat,cat,i16,i16,i16,i16
"""2017""","""2016""","""Dec""","""14 00490001532…","""15-DEC-2016""","""400,000""","""276,900""","""LW""","""276,900""","""0""","""TA_KBYRD""","""04-APR-2017""","""STROBEL, II GE…","""BECKMANN SCOTT…",null,"""860""",null,"""PEACHTREE""","""ST""",null,"""ATL""","""2301""",1,"""14004900015325…","""14 00490001532…","""9994""",860,null,"""PEACHTREE""","""ST""",null,"""ATL""","""R1""","""5""","""R3""","""106""","""1""","""2.680000000000…","""KA SPIRE RESID…","""5""","""BECKMANN SCOTT…","""BECKMANN RHOND…","""860""",null,null,"""PEACHTREE""","""ST""","""NE""","""GA""",null,"""2301""","""30308""",40200,236700,"""10""","""2005""",5,2,2,0
"""2012""","""2011""","""May""","""17 0076 LL304…","""26-MAY-2011""","""0""","""44,000""","""WD""","""58400""","""T""","""TA_KJORDAN""","""18-AUG-2011""","""BRACHA SAAR""","""BRACHA HOLDING…",null,"""8305""",null,"""SANTA FE""","""PKY""",null,"""FUL""",null,1,"""170076LL3040""","""17 0076 LL304…","""77985""",8305,null,"""SANTA FE""","""PKY""",null,"""SANDY SPRINGS""","""AC""","""59""","""R3""","""106""","""1""","""2.489999999999…","""KA VICTORIA HE…","""59""","""BRACHA HOLDING…",null,"""245""",null,null,"""WOODSCAPE""","""CT""",null,"""GA""",null,null,"""30022""",5200,36900,"""10""","""1991""",4,2,2,0
"""2022""","""2021""","""Jun""","""17 0032 LL505…","""15-JUN-2021""","""449,500""","""413,900""","""LW""","""413,900""","""0""","""TA_LWILLIA""","""10-NOV-2021""","""RONKART PIERRE…","""LUCAS LONNIE""",null,"""7500""",null,"""GLISTEN""","""AVE""",null,"""SANDY SPRINGS""","""317""",1,"""170032LL5055""","""17 0032 LL505…","""1781T""",7500,null,"""GLISTEN""","""AVE""",null,"""SANDY SPRINGS""","""RM3""","""59""","""R3""","""107""","""1""","""0.03""",null,"""59""","""LUCAS LONNIE""",null,"""7500""",null,null,"""GLISTEN""","""AVE""",null,"""GA""",null,null,"""30328""",57800,356100,"""7""","""2013""",6,3,2,1
"""2021""","""2020""","""Feb""","""14 02080010044…","""13-FEB-2020""","""227,500""","""220,700""","""LW""","""220,700""","""0""","""TA_SPOYTHR""","""05-NOV-2020""","""MICHIGAN INVES…","""LUMIERE MISCHA…",null,"""638""",null,"""ALBERT""","""ST""","""NW""","""ATLANTA""",null,1,"""14020800100447…","""14 02080010044…","""14781""",638,null,"""ALBERT""","""ST""","""NW""","""ATLANTA""","""R4""","""5""","""R3""","""101""","""1""","""0.172199999999…","""COLLIER HEIGHT…","""5""","""LUMIERE MISCHA…",null,"""50""",null,null,"""HURT""","""PLZ""",null,"""GA""",null,"""1444""","""30303""",26500,194200,"""2""","""1950""",8,4,2,0


In [9]:
unmatched = len(full_sales_parcel.filter(pl.col('parid_right').is_null()))
print('Total row count: ', len(full_sales_parcel))
print('Total unmatched parcels: ', unmatched)
print('Percent of total: ', unmatched / len(full_sales_parcel) * 100)

Total row count:  480453
Total unmatched parcels:  15703
Percent of total:  3.2683738055543414


In [10]:
unique_sales = len(full_sales_parcel.select(['parid', 'taxyr']).unique())
print('Total unique [parid, taxyr] keys: ', unique_sales)
print('Total non-unique [parid, taxyr] keys: ', len(full_sales_parcel) - unique_sales)
print('Percent (non-unique) of total: ', (len(full_sales_parcel) - unique_sales) / len(full_sales_parcel) * 100)

Total unique [parid, taxyr] keys:  380731
Total non-unique [parid, taxyr] keys:  99722
Percent (non-unique) of total:  20.7558283536579


Compared to original sales

In [11]:
all_sales = [read_file(sales_path + file, sales_dtypes, sales_new_names) for file in sales_files]
all_sales_append = pl.concat(all_sales, how='diagonal')
all_sales_append.write_csv('../output/sales/sales_all_years.csv')

Reading file:  C:/Users/nicho/Documents/research/FCS/data/sales/Sales2011.txt
Reading file:  C:/Users/nicho/Documents/research/FCS/data/sales/Sales2012.txt
Reading file:  C:/Users/nicho/Documents/research/FCS/data/sales/Sales2013.txt
Reading file:  C:/Users/nicho/Documents/research/FCS/data/sales/Sales2014.txt
Reading file:  C:/Users/nicho/Documents/research/FCS/data/sales/Sales2015.txt
Reading file:  C:/Users/nicho/Documents/research/FCS/data/sales/Sales2016.txt
Reading file:  C:/Users/nicho/Documents/research/FCS/data/sales/Sales2017.txt
Reading file:  C:/Users/nicho/Documents/research/FCS/data/sales/Sales2018.txt
Reading file:  C:/Users/nicho/Documents/research/FCS/data/sales/Sales2019.txt
Reading file:  C:/Users/nicho/Documents/research/FCS/data/sales/Sales2020.txt
Reading file:  C:/Users/nicho/Documents/research/FCS/data/sales/Sales2021.txt
Reading file:  C:/Users/nicho/Documents/research/FCS/data/sales/Sales2022.txt


In [12]:
add_rows = len(full_sales_parcel) - len(all_sales_append)
print('Additional non-unique rows generated from processing: ', add_rows)
print('Percent of original sales: ', add_rows / len(all_sales_append) * 100)

Additional non-unique rows generated from processing:  3215
Percent of original sales:  0.6736680649906336


### Post-Processing
---

Save before post-processing to avoid having to re-run entire process.

In [13]:
full_sales_parcel.write_csv('../output/merged/pre_sales_parcel_left.csv')

Convert salesprice, fmv, costval -> float  
Convert own_adrno, own_zip -> int  
AND drop rows with bad entries for all of the above variables except costval

In [14]:
init_size = len(full_sales_parcel)
full_sales_parcel = full_sales_parcel.select(
    pl.col('salesprice').str.replace_all(',','').cast(pl.Float64),
    pl.col('fmv').str.replace_all(',','').cast(pl.Float64),
    pl.col('costval').str.replace_all(',','').cast(pl.Float64),
    pl.col('*').exclude(['salesprice', 'fmv', 'costval'])
)

Number of null own_adrno.

In [81]:
len(full_sales_parcel.filter(
    pl.col('own_adrno').is_null()
))

35145

Number of rows where both own_adrno and own_adrstr are null. We want count of null own_adrno = count of null own_adrstr, since we can assume there is no own_adrno if there is no street information either.

In [83]:
len(full_sales_parcel.filter(
    pl.col('own_adrno').is_null() &
    pl.col('own_adrstr').is_null()
))

17141

In [87]:
full_sales_parcel.filter(
    pl.col('own_adrno').is_null()
)[['own_adrno', 'own_adrstr']].sample(2)

own_adrno,own_adrstr
str,str
null,"""P O BOX 650043…"
null,"""PO BOX 6993"""


Extract own_adrno from PO boxes to reduce the number of nulls. Most null own_adrno are caused by PO boxes.

In [103]:
full_sales_parcel = full_sales_parcel.with_columns(
    pl.when(pl.col('own_adrno').is_null())
    .then(pl.col('own_adrstr').str.extract(r'(\d+)'))
    .otherwise(pl.col('own_adrno'))
    .alias('own_adrno')
)

Now we can drop nulls for own_adrno. We can also drop nulls for own_zip.

In [116]:
init_len = len(full_sales_parcel)
full_sales_parcel = full_sales_parcel.drop_nulls(subset=['own_adrno', 'own_zip'])
print('Number of rows dropped: ', init_len - len(full_sales_parcel))
print('Percent: ', (init_len - len(full_sales_parcel)) / len(full_sales_parcel) * 100)

Number of rows dropped:  18217
Percent:  3.9410604106992966


Investigate incorrect own_zip.

In [117]:
full_sales_parcel.filter(
    pl.col('own_zip').str.contains(r'[a-zA-Z]')
)['own_zip'].sample(5)

own_zip
str
"""L7M-4"""
"""L5M 5"""
"""L4E4B"""
"""NSW 2"""
"""EL61L"""


Number of incorrect own_zip.

In [118]:
len(full_sales_parcel.filter(
    pl.col('own_zip').str.contains(r'[a-zA-Z]')
))

165

In [124]:
len(full_sales_parcel)

462236

Drop incorrect own_zip.

In [133]:
init_len = len(full_sales_parcel)

full_sales_parcel = full_sales_parcel.with_columns(
    pl.when(pl.col('own_zip').str.contains(r'[a-zA-Z]'))
    .then(None)
    .otherwise(pl.col('own_zip'))
    .alias('own_zip')
).drop_nulls(['own_zip'])

print('Count dropped: ', init_len - len(full_sales_parcel))

Count dropped:  165


Cast own_adrno and own_zip -> int.

In [134]:
full_sales_parcel = full_sales_parcel.select(
    pl.col('own_adrno').cast(pl.Int32),
    pl.col('own_zip').cast(pl.Int32),
    pl.col('*').exclude(['own_adrno', 'own_zip'])
)

In [135]:
full_sales_parcel.sample(5)

own_adrno,own_zip,salesprice,fmv,costval,taxyr,sale_year,sale_month,parid,sale_date,deed,saleval,appraiser,when,grantor,grantee,sale_adrpre,sale_adrno,sale_adrdir,sale_adrstr,sale_adrsuf,sale_adrsuf2,sale_cityname,sale_unitno,livunit,parid_strip,parid_right,nbhd,site_adrno,site_adrdir,site_adrstr,site_adrsuf,site_adrsuf2,site_cityname,zoning,site_muni,class,luc,livunit_right,calcacres,note1,taxdist,own1,own2,own_adradd,own_adrdir,own_adrstr,own_adrsuf,own_adrsuf2,own_statecode,own_country,own_unitno,aprland,aprbldg,style,yrblt,rmtot,rmbed,fixbath,fixhalf
i32,i32,f64,f64,f64,cat,cat,cat,str,str,cat,cat,str,str,str,str,cat,str,cat,str,cat,cat,cat,str,i16,str,str,cat,i64,str,str,str,str,cat,cat,cat,cat,cat,str,str,str,cat,str,str,str,cat,str,cat,str,cat,cat,str,i32,i32,cat,cat,i16,i16,i16,i16
3311,30324,1.199e6,894200.0,894200.0,"""2021""","""2020""","""Dec""","""17 00080001004…","""31-DEC-2020""","""LW""","""0""","""TA_JCHRIST""","""30-JUN-2021""","""CHASE HOMES IN…","""YOUNG TRACEY A…",null,"""3311""",null,"""WEST ROXBORO""","""RD""","""NE""","""ATLANTA""",null,1,"""17000800010047…","""17 00080001004…","""171102""",3311,null,"""WEST ROXBORO""","""RD""","""NE""","""ATLANTA""","""R3""","""5""","""R3""","""101""","""1""","""0.5292""","""*DATA VERIFIED…","""5""","""YOUNG TRACEY A…",null,null,"""W""","""ROXBORO""","""RD""","""NE""","""GA""",null,null,215300,678900,"""1""","""2018""",10,5,3,1
240,30213,120000.0,119930.0,124200.0,"""2013""","""2012""","""May""","""07 27000169199…","""22-MAY-2012""","""WD""","""0""","""CLT""","""29-MAY-2013""","""DARNLEY MONIQU…","""MINTER BRANDI""",null,"""240""",null,"""BUCKINGHAM""","""LN""",null,"""FBN""",null,1,"""07270001691991…","""07 27000169199…","""726""",240,null,"""BUCKINGHAM""","""LN""",null,"""FAIRBURN""","""PD""","""25""","""R3""","""101""","""1""","""0.192""","""KA PARKS @ DUR…","""25""","""MINTER BRANDI""",null,null,null,"""BUCKINGHAM""","""LANE""",null,"""GA""",null,null,24630,95300,"""1""","""2006""",9,5,3,0
4171,30035,27000.0,27000.0,16600.0,"""2016""","""2015""","""Aug""","""14 01090003030…","""31-AUG-2015""","""WD""","""9""","""TA_JBANKS""","""12-FEB-2016""","""PABON NELSON A…","""HILL LAURENE T…",null,"""862""",null,"""BECKWITH""","""ST""","""SW""","""ATL""",null,1,"""14010900030305…","""14 01090003030…","""1415""",862,null,"""BECKWITH""","""ST""","""SW""","""ATL""","""R5""","""5""","""R3""","""101""","""1""","""9.64E-2""","""AN MEAS VERIFI…","""5""","""MAROON & WHITE…",null,null,null,"""SNAPFINGER WOO…","""DR""",null,"""GA""",null,null,8900,18100,"""1""","""1970""",6,2,1,0
3481,30326,206000.0,189500.0,189500.0,"""2019""","""2018""","""Dec""","""17 00090004119…","""17-DEC-2018""","""LW""","""0""","""TA_JCHRIST""","""20-FEB-2019""","""APONTE MIQUEL …","""SHARMA ANJAVI …",null,"""3481""",null,"""LAKESIDE""","""DR""","""NE""","""ATLANTA""","""1205""",1,"""17000900041199…","""17 00090004119…","""17095""",3481,null,"""LAKESIDE""","""DR""","""NE""","""ATLANTA""","""PDH""","""5""","""R3""","""106""","""1""","""1.72E-2""","""KA GRANDVIEW -…","""5""","""SHARMA ANJAVI …",null,null,null,"""LAKESIDE""","""DR""","""NE""","""GA""",null,"""1205""",28400,161100,"""10""","""1990""",3,1,1,0
975,30319,1.35e6,300100.0,300100.0,"""2011""","""2010""","""May""","""17 00120003037…","""20-MAY-2010""","""WD""","""P""","""TA_ZMITCHE""","""01-APR-2011""","""HUBBARD JOHNNY…","""LINCH DAVID E""",null,"""4308""",null,"""CLUB""","""DR""",null,"""ATL""",null,0,"""17001200030379…","""17 00120003037…","""1711""",4308,null,"""CLUB""","""DR""",null,"""ATLANTA""","""R3""","""5""","""R4""","""100""","""0""","""1.68""","""HOUSE DEMO 12/…","""5""","""LINCH DAVID E""",null,null,null,"""WINALL DOWN""","""RD""",null,"""GA""",null,null,285500,14600,"""2""","""1950""",9,5,3,0


### Final output
---

In [136]:
full_sales_parcel.write_csv('../output/merged/sales_parcel_left.csv')